# Tutorial

This notebook will show how to use the `leftcorner` library to efficiently remove left recursion from a weighted context-free grammar.  Along the way, we will demonstrate how to visualize and work with derivation trees.

We will assume some basic familarity with [our paper](https://arxiv.org/abs/2311.16258).

In [1]:
from leftcorner import CFG, Real
from leftcorner.misc import Latex, display_table

We start by creating a simple weighted context-free grammar with real-valued weights.

In [2]:
cfg = CFG.from_string("""
1:  S     -> NP VP     
.5: NP    -> PossP NN 
.2: PossP -> NP 's    
.4: NP    -> my sister
1:  NN    -> diploma   
.1: VP    -> arrived   

""", Real)

In [3]:
cfg

1.0: S → NP VP
0.5: NP → PossP NN
0.2: PossP → NP 's
0.4: NP → my sister
1.0: NN → diploma
0.1: VP → arrived

In [4]:
print(f'size: {cfg.size}, num rules: {cfg.num_rules}')

size: 16, num rules: 6


Below we show all derivations in `cfg` with height $\le T$ in order of increasing weight.

In [5]:
T = 6
D = sorted(cfg.derivations(cfg.S, T), key=lambda d: -d.weight().score)

In [6]:
for d in D:
    print(f'{d.weight().score:.5f}', d.Yield())
    display(d)

0.04000 ('my', 'sister', 'arrived')


(S (NP my sister) (VP arrived))

0.00400 ('my', 'sister', "'s", 'diploma', 'arrived')


(S (NP (PossP (NP my sister) 's) (NN diploma)) (VP arrived))

0.00040 ('my', 'sister', "'s", 'diploma', "'s", 'diploma', 'arrived')


(S (NP (PossP (NP (PossP (NP my sister) 's) (NN diploma)) 's) (NN diploma)) (VP arrived))

If we try to parse the string `my sister 's diploma arrived` using a top-down parser, we will encounter a `RecursionError`:

In [7]:
s = "my sister 's diploma arrived".split()
try:
    for d in cfg.derivations_of(s):
        print(d)
except RecursionError:
    print('Recursion error 🤔')

Recursion error 🤔


This is because the grammar is left recursive, as is shown by the following assertion.

In [8]:
assert cfg.is_left_recursive()

We can see this by analyzing `cfg`'s the left-recursion graph (shown below).  It has a cycle from `NP` to `PossP`.

In [9]:
G = cfg.left_recursion_graph()
G

The simplest way to remove left recursion is to call the `elim_left_recursion` method.

In [10]:
cfg_lr = cfg.elim_left_recursion()

In [11]:
assert not cfg_lr.is_left_recursive()

We can now use our top-down derivation enumeration method.

In [12]:
for d in cfg_lr.derivations_of(s):
    display(d)

(S (~S (NP (~NP my sister) (NP/NP 's (NP/PossP (NN (~NN diploma)) (NP/NP )))) (VP (~VP arrived))))

We can remove the empty constituents in the tree with the `nullaryremove` method.

In [13]:
for d in cfg_lr.nullaryremove().derivations_of(s):
    display(d)

(S (~S (NP (~NP my sister) (NP/NP 's (NP/PossP (NN (~NN diploma))))) (VP (~VP arrived))))

## Lower-level transformations

Some readers may be interestined in exploring the lower-level left-corner transformation methods.  Let's apply our generalized left-corner transformation (`lc_generalized`) on it. We select the parameters ($\overline{R}$ and $\overline{C}$) according to our recipe from the paper (see Theorem 4).

In [14]:
Rbar = cfg.find_lr_rules()
Cbar = cfg.sufficient_Xs(Rbar)

display(Latex(r"""
$\overline{R}=$ $\{{\texttt %s\}}$
""" % (Rbar)))
display(Latex(r"""
$\overline{C}=$ $\{{\texttt %s\}}$
""" % (Cbar)))

lcfg = cfg.lc_generalized(Cbar, Rbar, filter=False)
assert not lcfg.is_left_recursive()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Left recursion is gone! But the grammar is quite large... Let's take care of that using trimming.

In [15]:
print(lcfg.size, lcfg.num_rules)
nlcfg = lcfg.trim()
print(nlcfg.size, nlcfg.num_rules)

76 34
26 11


Better yet - we can use our filtering tricks during the transformation itself. This eliminates most of the useless production rules and the transformation will run much more efficiently on large grammars.

In [16]:
lcfg2 = cfg.lc_generalized(Cbar, Rbar, filter=True)
assert not lcfg2.is_left_recursive()
print(lcfg2.size, lcfg2.num_rules)

29 12


There is no issue with left recursion anymore, we can display our desired tree:

In [17]:
for d in nlcfg.derivations_of(s):
    display(d)

(S (~S (NP (~NP my sister) (NP/NP 's (NP/PossP (NN (~NN diploma)) (NP/NP )))) (VP (~VP arrived))))

You may note that the above tree looks a little different from the tree in Fig. 1 of the paper. That is because that figure used an additional production rules in $\overline{R}$, `S → NP VP`. Let's replicate it.

In [18]:
cfg

1.0: S → NP VP
0.5: NP → PossP NN
0.2: PossP → NP 's
0.4: NP → my sister
1.0: NN → diploma
0.1: VP → arrived

In [19]:
Rbar = cfg.find_lr_rules()
Cbar = cfg.sufficient_Xs(Rbar)

Rbar.add(cfg.rules[0])

display(Latex(r"""
$\overline{R}=$ $\{{\texttt %s\}}$
""" % (Rbar)))
display(Latex(r"""
$\overline{C}=$ $\{{\texttt %s\}}$
""" % (Cbar)))

lcfg = cfg.lc_generalized(Cbar, Rbar, filter=True)
assert not lcfg.is_left_recursive()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [20]:
for d in lcfg.derivations_of(s):
    display(d)

(S (~NP my sister) (S/NP 's (S/PossP (NN (~NN diploma)) (S/NP (VP (~VP arrived)) (S/S )))))

Below, we use the unfold transformation to remove the unnecessary frozen recovery rules for `VP` and `NN` to get a slightly smaller tree (and grammar).

In [21]:
lcfg

1: S/S → 
1.0: S/NP → VP S/S
0.5: S/PossP → NN S/NP
0.5: ~NP → ~PossP NN
0.2: S/NP → 's S/PossP
0.4: ~NP → my sister
1.0: ~NN → diploma
0.1: ~VP → arrived
1: S → ~S
1: NN → ~NN
1: VP → ~VP
1: S → ~NP S/NP

In [22]:
lcfg_u = (lcfg
    .unfold(8, 0)
    .unfold(8, 0)
).trim()
lcfg_u

1: S/S → 
1.0: S/NP → VP S/S
0.5: S/PossP → NN S/NP
0.2: S/NP → 's S/PossP
0.4: ~NP → my sister
0.1: ~VP → arrived
1: VP → ~VP
1: S → ~NP S/NP
1.0: NN → diploma

In [23]:
for d in lcfg_u.derivations_of(s):
    display(d)

(S (~NP my sister) (S/NP 's (S/PossP (NN diploma) (S/NP (VP (~VP arrived)) (S/S )))))

Our library also provides nullary rule elimination. (As well as unary rule removal and transformation to CNF.)

In [24]:
for d in lcfg_u.nullaryremove().derivations_of(s):
    display(d)

(S (~NP my sister) (S/NP 's (S/PossP (NN diploma) (S/NP (VP (~VP arrived))))))

### Comparing Basic, Selective, and Generalized Left-Corner Transformations

As discussed in the paper, our transformation is a generalization of the basic and selective left-corner transformations. Let's compare them on our example grammar.

In [25]:
basic = cfg.lc_generalized(cfg.N | cfg.V, cfg.rules)
slct = cfg.lc_generalized(cfg.N | cfg.V, cfg.find_lr_rules())
glct = cfg.lc_generalized(cfg.sufficient_Xs(cfg.find_lr_rules()), cfg.find_lr_rules())

Our glct gives a smaller grammar size in comparison.

In [26]:
display_table([
    (basic, slct, glct)
], headings=['basic', 'selective', 'generalized'])

basic,selective,generalized
1: S/S → 1: NN/NN → 1: VP/VP → 1.0: S/NP → VP S/S 0.5: S/PossP → NN S/NP 0.2: S/NP → 's S/PossP 0.4: S/my → sister S/NP 1.0: NN/diploma → NN/NN 0.1: VP/arrived → VP/VP 1: S → my S/my 1: S → ~NP S/NP 1: VP → arrived VP/arrived 1: S → ~PossP S/PossP 1: NN → diploma NN/diploma 1: S → ~S S/S 1: NN → ~NN NN/NN 1: VP → ~VP VP/VP,1: NP/NP → 1: S/S → 1: NN/NN → 1: VP/VP → 1.0: ~S → NP VP 0.5: NP/PossP → NN NP/NP 0.2: NP/NP → 's NP/PossP 0.4: ~NP → my sister 1.0: ~NN → diploma 0.1: ~VP → arrived 1: NP → ~NP NP/NP 1: NP → ~PossP NP/PossP 1: S → ~S S/S 1: NN → ~NN NN/NN 1: VP → ~VP VP/VP,1: NP/NP → 1.0: ~S → NP VP 0.5: NP/PossP → NN NP/NP 0.5: ~NP → ~PossP NN 0.2: NP/NP → 's NP/PossP 0.4: ~NP → my sister 1.0: ~NN → diploma 0.1: ~VP → arrived 1: S → ~S 1: NN → ~NN 1: VP → ~VP 1: NP → ~NP NP/NP


In [27]:
display_table([(d, basic.mapping(d), slct.mapping(d), glct.mapping(d)) for d in D], headings=['original', 'basic', 'selective', 'generalized'])

### Comparing GLCT and Speculation

Lastly, our tutorial wouldn't be complete without a comparison with speculation. Below, we show the original, speculation, and glct derivations side by side.

In [28]:
speculation = cfg.speculate(cfg.N, cfg.rules)

display_table([[d, speculation.mapping(d), glct.mapping(d)] for d in D], ['original', 'speculation', 'GLCT'])

## Semiring Weights

Our library supports commutative semirings.  We have included a few example semirings in the library, such as `Real`, `Boolean`, `MaxPlus`, `MaxTimes`, `Log`, and `Entropy`.

In [29]:
from leftcorner.semiring import Boolean, MaxPlus, MaxTimes, Log, Entropy

Below are some examples of how to create grammars that use these semirings for their weights.

In [30]:
CFG.from_string("""
True: S     -> NP VP     
True: NP    -> my sister
True: VP    -> arrived   
""", Boolean)

True: S → NP VP
True: NP → my sister
True: VP → arrived

In [31]:
CFG.from_string("""
1:  S     -> NP VP     
.4: NP    -> my sister
.1: VP    -> arrived   
""", MaxTimes)

MaxTimes(1.0): S → NP VP
MaxTimes(0.4): NP → my sister
MaxTimes(0.1): VP → arrived